# Import data

In [1]:
import pandas as pd

df = pd.read_csv('./data/1_600_000_random_tweets.csv')
df.Sentiment = df.Sentiment.apply(lambda x: 0 if x==0 else 1)
df.head()

,Sentiment,Text
0,0,"switchfoot - awww, that's a bummer. you shou..."
1,0,what should i get pops 4 fathers day? suggesti...
2,1,i like socks again
3,1,angiemac78 had to close it down for a while. u...
4,0,is upset that he can't update his facebook by ...


In [2]:
%pip install -U tensorflow-hub tensorflow-text

Requirement already up-to-date: tensorflow-hub in ./venv/lib/python3.8/site-packages (0.12.0)
Requirement already up-to-date: tensorflow-text in ./venv/lib/python3.8/site-packages (2.8.2)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub


bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2022-05-05 11:05:41.496199: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.


In [9]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [10]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [12]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


X_train, X_test, y_train, y_test = train_test_split(df.Text.values, df.Sentiment.values,test_size=0.33,shuffle=True)

history = model.fit(X_train,y_train,epochs=5, batch_size=128, shuffle=True,validation_split=0.3)


Epoch 1/5
   3/5863 [..............................] - ETA: 80:14:52 - loss: 0.7308 - accuracy: 0.5260 - precision: 0.4757 - recall: 0.5087

In [ ]:
fig, ax1  = plt.subplots(1, 1,figsize=(15,4))
ax1.plot(history.history['root_mean_squared_error'],label='Loss')
ax1.plot(history.history['val_root_mean_squared_error'],label=f'Validation Loss')
ax1.legend(loc='upper left')